 # Canada cumulative cases prediction
- I have collected timeseries data for cumulative cases in Canada from 2020/1/25 to 2021/7/30.
- In order to verify the accuracy of the prediction, I assume that today is June 30, 2021, and use different machine learning models to predict the cumulative number of cases in the next month (July).

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime as dt
from datetime import timedelta
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score,silhouette_samples
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error,r2_score
import statsmodels.api as sm
from statsmodels.tsa.api import Holt,SimpleExpSmoothing,ExponentialSmoothing
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import PolynomialFeatures

In [2]:
#load data
cases_canada=pd.read_csv(r"cases_timeseries\cases_timeseries_canada.csv")
cases_canada1=cases_canada[0:-30]
cases_canada1=cases_canada1.drop(["new_cases"],axis=1)

FileNotFoundError: [Errno 2] No such file or directory: 'cases_timeseries\\cases_timeseries_canada.csv'

In [ ]:
cases_canada1["Days Since"]=cases_canada1.index-cases_canada1.index[0]

In [ ]:
cases_canada1.info()

In [ ]:
#Linear Regression Model for Confirm Cases Prediction
train_ml=cases_canada1.iloc[:int(cases_canada1.shape[0]*0.95)]
valid_ml=cases_canada1.iloc[int(cases_canada1.shape[0]*0.95):]
lin_reg=LinearRegression(normalize=True)
model_scores=[]
lin_reg.fit(np.array(train_ml["Days Since"]).reshape(-1,1),np.array(train_ml["cumulative_cases"]).reshape(-1,1))
prediction_valid_linreg=lin_reg.predict(np.array(valid_ml["Days Since"]).reshape(-1,1))
model_scores.append(np.sqrt(mean_squared_error(valid_ml["cumulative_cases"],prediction_valid_linreg)))
print("Root Mean Square Error for Linear Regression: ",np.sqrt(mean_squared_error(valid_ml["cumulative_cases"],prediction_valid_linreg)))

In [ ]:
prediction_linreg=lin_reg.predict(np.array(cases_canada1["Days Since"]).reshape(-1,1))
linreg_output=[]
for i in range(prediction_linreg.shape[0]):
    linreg_output.append(prediction_linreg[i][0])

In [ ]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=cases_canada1.index, y=cases_canada1["cumulative_cases"],
                    mode='lines+markers',name="Train Data for Cumulative Cases"))
fig.add_trace(go.Scatter(x=cases_canada1.index, y=linreg_output,
                    mode='lines',name="Linear Regression Best Fit Line",
                    line=dict(color='black', dash='dot')))
fig.update_layout(title="Cumulative Cases Linear Regression Prediction",
                 xaxis_title="Days since",yaxis_title="Confirmed Cases",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

In [ ]:
#Polynomial Regression for Prediction of Cumulative Cases
poly = PolynomialFeatures(degree = 8)
train_poly=poly.fit_transform(np.array(train_ml["Days Since"]).reshape(-1,1))
valid_poly=poly.fit_transform(np.array(valid_ml["Days Since"]).reshape(-1,1))
y=train_ml["cumulative_cases"]
linreg=LinearRegression(normalize=True)
linreg.fit(train_poly,y)
prediction_poly=linreg.predict(valid_poly)
rmse_poly=np.sqrt(mean_squared_error(valid_ml["cumulative_cases"],prediction_poly))
model_scores.append(rmse_poly)
print("Root Mean Squared Error for Polynomial Regression: ",rmse_poly)

In [ ]:
comp_data=poly.fit_transform(np.array(cases_canada1["Days Since"]).reshape(-1,1))
predictions_poly=linreg.predict(comp_data)
plt.figure(figsize=(11,6))
fig=go.Figure()
fig.add_trace(go.Scatter(x=cases_canada1.index, y=cases_canada1["cumulative_cases"],
                    mode='lines+markers',name="Train Data for Cumulative cases"))
fig.add_trace(go.Scatter(x=cases_canada1.index, y=predictions_poly,
                    mode='lines',name="Polynomial Regression Best Fit",
                    line=dict(color='black', dash='dot')))
fig.update_layout(title="Cumulative cases Polynomial Regression Prediction",
                 xaxis_title="Days since",yaxis_title="Cumulative_cases",
                 legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

In [ ]:
#SVR Model for Prediction of Cumulative Cases
svm=SVR(C=1,degree=6,kernel='poly',epsilon=0.01)
#Fitting model on the training data
svm.fit(np.array(train_ml["Days Since"]).reshape(-1,1),np.array(train_ml["cumulative_cases"]).reshape(-1,1))
prediction_valid_svm=svm.predict(np.array(valid_ml["Days Since"]).reshape(-1,1))
prediction_svm=svm.predict(np.array(cases_canada1["Days Since"]).reshape(-1,1))
model_scores.append(np.sqrt(mean_squared_error(valid_ml["cumulative_cases"],prediction_valid_svm)))
print("Root Mean Square Error for Support Vectore Machine: ",np.sqrt(mean_squared_error(valid_ml["cumulative_cases"],prediction_valid_svm)))

In [ ]:
plt.figure(figsize=(11,6))
prediction_svm=svm.predict(np.array(cases_canada1["Days Since"]).reshape(-1,1))
fig=go.Figure()
fig.add_trace(go.Scatter(x=cases_canada1.index, y=cases_canada1["cumulative_cases"],
                    mode='lines+markers',name="Train Data for Cumulative Cases"))
fig.add_trace(go.Scatter(x=cases_canada1.index, y=prediction_svm,
                    mode='lines',name="Support Vector Machine Best fit Kernel",
                    line=dict(color='black', dash='dot')))
fig.update_layout(title="Cumulative Cases Support Vectore Machine Regressor Prediction",
                 xaxis_title="Days since",yaxis_title="Cumulative Cases",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

In [ ]:
#create prediction table for next month
new_date=[]
new_prediction_lr=[]
new_prediction_svm=[]
new_prediction_poly=[]
for i in range(1,31):
    new_prediction_lr.append(lin_reg.predict(np.array(cases_canada1["Days Since"].max()+i).reshape(-1,1))[0][0])
    new_prediction_svm.append(svm.predict(np.array(cases_canada1["Days Since"].max()+i).reshape(-1,1))[0])
    new_date_poly=poly.fit_transform(np.array(cases_canada1["Days Since"].max()+i).reshape(-1,1))
    new_prediction_poly.append(linreg.predict(new_date_poly)[0])
new_date = pd.date_range("20210701","20210730")

In [ ]:
pd.set_option('display.float_format', lambda x: '%.6f' % x)
model_predictions=pd.DataFrame(zip(new_date,new_prediction_lr,new_prediction_poly,new_prediction_svm),
                               columns=["Dates","Linear Regression Prediction","Polynonmial Regression Prediction","SVM Prediction"])
model_predictions.head()

In [ ]:
#Time Series Forecasting
#Holt's Linear Model
model_train=cases_canada1.iloc[:int(cases_canada1.shape[0]*0.95)]
valid=cases_canada1.iloc[int(cases_canada1.shape[0]*0.95):]
y_pred=valid.copy()
holt=Holt(np.asarray(model_train["cumulative_cases"])).fit(smoothing_level=0.4, smoothing_slope=0.4,optimized=False)
y_pred["Holt"]=holt.forecast(len(valid))
model_scores.append(np.sqrt(mean_squared_error(y_pred["cumulative_cases"],y_pred["Holt"])))
print("Root Mean Square Error Holt's Linear Model: ",np.sqrt(mean_squared_error(y_pred["cumulative_cases"],y_pred["Holt"])))
holt_new_prediction=[]
for i in range(1,31):
    holt_new_prediction.append(holt.forecast((len(valid)+i))[-1])
model_predictions["Holt's Linear Model Prediction"]=holt_new_prediction
model_predictions.head()

In [ ]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=model_train.index, y=model_train["cumulative_cases"],
                    mode='lines+markers',name="Train Data for Cumulative Cases"))
fig.add_trace(go.Scatter(x=valid.index, y=valid["cumulative_cases"],
                    mode='lines+markers',name="Validation Data for Confirmed Cases",))
fig.add_trace(go.Scatter(x=valid.index, y=y_pred["Holt"],
                    mode='lines+markers',name="Prediction of Cumulative Cases",))
fig.update_layout(title="Cumulative Cases Holt's Linear Model Prediction",
                 xaxis_title="Days since",yaxis_title="Cumulative Cases",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

In [ ]:
#Holt's Winter Model
es=ExponentialSmoothing(np.asarray(model_train['cumulative_cases']),seasonal_periods=14,trend='add', seasonal='mul').fit()
y_pred["Holt's Winter Model"]=es.forecast(len(valid))
model_scores.append(np.sqrt(mean_squared_error(y_pred["cumulative_cases"],y_pred["Holt's Winter Model"])))
print("Root Mean Square Error for Holt's Winter Model: ",np.sqrt(mean_squared_error(y_pred["cumulative_cases"],y_pred["Holt's Winter Model"])))
holt_winter_new_prediction=[]
for i in range(1,31):
    holt_winter_new_prediction.append(es.forecast((len(valid)+i))[-1])
model_predictions["Holt's Winter Model Prediction"]=holt_winter_new_prediction
model_predictions.head()

In [ ]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=model_train.index, y=model_train["cumulative_cases"],
                    mode='lines+markers',name="Train Data for Cumulative Cases"))
fig.add_trace(go.Scatter(x=valid.index, y=valid["cumulative_cases"],
                    mode='lines+markers',name="Validation Data for Cumulative Cases",))
fig.add_trace(go.Scatter(x=valid.index, y=y_pred["Holt\'s Winter Model"],
                    mode='lines+markers',name="Prediction of Cumulative Cases",))
fig.update_layout(title="Cumulative Cases Holt's Winter Model Prediction",
                 xaxis_title="Date",yaxis_title="Cumulative Cases",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

In [ ]:
#AR Model (using AUTO ARIMA)
from pmdarima.arima import auto_arima
model_ar= auto_arima(model_train["cumulative_cases"],trace=True, error_action='ignore', start_p=0,start_q=0,max_p=4,max_q=0,
                   suppress_warnings=True,stepwise=False,seasonal=False)
model_ar.fit(model_train["cumulative_cases"])

In [ ]:
prediction_ar=model_ar.predict(len(valid))
y_pred["AR Model Prediction"]=prediction_ar
model_scores.append(np.sqrt(mean_squared_error(y_pred["cumulative_cases"],y_pred["AR Model Prediction"])))
print("Root Mean Square Error for AR Model: ",np.sqrt(mean_squared_error(y_pred["cumulative_cases"],y_pred["AR Model Prediction"])))

In [ ]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=model_train.index, y=model_train["cumulative_cases"],
                    mode='lines+markers',name="Train Data for Cumulative Cases"))
fig.add_trace(go.Scatter(x=valid.index, y=valid["cumulative_cases"],
                    mode='lines+markers',name="Validation Data for Cumulative Cases",))
fig.add_trace(go.Scatter(x=valid.index, y=y_pred["AR Model Prediction"],
                    mode='lines+markers',name="Prediction of Cumulative Cases",))
fig.update_layout(title="Cumulative Cases AR Model Prediction",
                 xaxis_title="Days since",yaxis_title="Cumulative Cases",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

In [ ]:
AR_model_new_prediction=[]
for i in range(1,31):
    AR_model_new_prediction.append(model_ar.predict(len(valid)+i)[-1])
model_predictions["AR Model Prediction"]=AR_model_new_prediction
model_predictions.head()

In [ ]:
#MA Model (using AUTO ARIMA)
model_train=cases_canada1.iloc[:int(cases_canada1.shape[0]*0.95)]
valid=cases_canada1.iloc[int(cases_canada1.shape[0]*0.95):]
y_pred=valid.copy()
model_ma= auto_arima(model_train["cumulative_cases"],trace=True, error_action='ignore', start_p=0,start_q=0,max_p=0,max_q=2,
                   suppress_warnings=True,stepwise=False,seasonal=False)
model_ma.fit(model_train["cumulative_cases"])

In [ ]:
prediction_ma=model_ma.predict(len(valid))
y_pred["MA Model Prediction"]=prediction_ma
model_scores.append(np.sqrt(mean_squared_error(valid["cumulative_cases"],prediction_ma)))
print("Root Mean Square Error for MA Model: ",np.sqrt(mean_squared_error(valid["cumulative_cases"],prediction_ma)))
MA_model_new_prediction=[]
for i in range(1,31):
    MA_model_new_prediction.append(model_ma.predict(len(valid)+i)[-1])
model_predictions["MA Model Prediction"]=MA_model_new_prediction
model_predictions.head()

In [ ]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=model_train.index, y=model_train["cumulative_cases"],
                    mode='lines+markers',name="Train Data for Cumulative Cases"))
fig.add_trace(go.Scatter(x=valid.index, y=valid["cumulative_cases"],
                    mode='lines+markers',name="Validation Data for Cumulative Cases",))
fig.add_trace(go.Scatter(x=valid.index, y=y_pred["MA Model Prediction"],
                    mode='lines+markers',name="Prediction for Cumulative Cases",))
fig.update_layout(title="Cumulative Cases MA Model Prediction",
                 xaxis_title="Date",yaxis_title="Cumulative Cases",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

In [ ]:
#ARIMA Model (using AUTOARIMA)
model_train=cases_canada1.iloc[:int(cases_canada1.shape[0]*0.95)]
valid=cases_canada1.iloc[int(cases_canada1.shape[0]*0.95):]
y_pred=valid.copy()
model_arima= auto_arima(model_train["cumulative_cases"],trace=True, error_action='ignore', start_p=1,start_q=1,max_p=3,max_q=3,
                   suppress_warnings=True,stepwise=False,seasonal=False)
model_arima.fit(model_train["cumulative_cases"])
prediction_arima=model_arima.predict(len(valid))
y_pred["ARIMA Model Prediction"]=prediction_arima
model_scores.append(np.sqrt(mean_squared_error(valid["cumulative_cases"],prediction_arima)))
print("Root Mean Square Error for ARIMA Model: ",np.sqrt(mean_squared_error(valid["cumulative_cases"],prediction_arima)))

In [ ]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=model_train.index, y=model_train["cumulative_cases"],
                    mode='lines+markers',name="Train Data for Cumulative Cases"))
fig.add_trace(go.Scatter(x=valid.index, y=valid["cumulative_cases"],
                    mode='lines+markers',name="Validation Data for Cumulative Cases",))
fig.add_trace(go.Scatter(x=valid.index, y=y_pred["ARIMA Model Prediction"],
                    mode='lines+markers',name="Prediction for Cumulative Cases",))
fig.update_layout(title="Cumulative Cases ARIMA Model Prediction",
                 xaxis_title="Date",yaxis_title="Cumulative Cases",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

In [ ]:
# SARIMA Model (using AUTO ARIMA)
model_sarima= auto_arima(model_train["cumulative_cases"],trace=True, error_action='ignore',start_p=0,start_q=0,max_p=2,max_q=2,m=7,suppress_warnings=True,stepwise=True,seasonal=True)
model_sarima.fit(model_train["cumulative_cases"])
prediction_sarima=model_sarima.predict(len(valid))
y_pred["SARIMA Model Prediction"]=prediction_sarima
model_scores.append(np.sqrt(mean_squared_error(y_pred["cumulative_cases"],y_pred["SARIMA Model Prediction"])))
print("Root Mean Square Error for SARIMA Model: ",np.sqrt(mean_squared_error(y_pred["cumulative_cases"],y_pred["SARIMA Model Prediction"])))

In [ ]:
SARIMA_model_new_prediction=[]
for i in range(1,31):
    SARIMA_model_new_prediction.append(model_sarima.predict(len(valid)+i)[-1])
model_predictions["SARIMA Model Prediction"]=SARIMA_model_new_prediction
model_predictions.head()

In [ ]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=model_train.index, y=model_train["cumulative_cases"],
                    mode='lines+markers',name="Train Data for Cumulative Cases"))
fig.add_trace(go.Scatter(x=valid.index, y=valid["cumulative_cases"],
                    mode='lines+markers',name="Validation Data for Cumulative Cases",))
fig.add_trace(go.Scatter(x=valid.index, y=y_pred["SARIMA Model Prediction"],
                    mode='lines+markers',name="Prediction for Cumulative Cases",))
fig.update_layout(title="Cumulative Cases SARIMA Model Prediction",
                 xaxis_title="Days since",yaxis_title="Cumulative Cases",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

In [ ]:
#Summarization of Forecasts using different Models
model_names=["Linear Regression","Polynomial Regression","Support Vector Machine Regressor","Holt's Linear Regression","Holt's Winter Model",
            "Auto Regressive Model (AR)","Moving Average Model (MA)","ARIMA Model","SARIMA Model"]
model_summary=pd.DataFrame(zip(model_names,model_scores),columns=["Model Name","Root Mean Squared Error"]).sort_values(["Root Mean Squared Error"])
model_summary

# Conclusion
- After testing nine models above to predict the cumulatives cases in Canada in July, I came to the conclusion that **SARIMA Model** has the least Root Mean Squared Error, so it is the best model for us to predict the cumulatives cases in Canada in the future.
- With regard to other countries, I will continue to test the prediction accuracy of these nine models to choose the best model to predict the future.

In [ ]:
model_predictions["SARIMA Model Prediction"]